<a href="https://colab.research.google.com/github/Arpitsinghvampire/PyTorch_codes/blob/main/INTRODUCTION_TO_PYTORCH_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
x = torch.tensor([[-1,-1] , [-2,-2]]) #two dimensional vector as the tensor
x

tensor([[-1, -1],
        [-2, -2]])

In [2]:
x = torch.tensor(7) #scalara as the tensor
x

tensor(7)

In [3]:
#now lets create a tensor of a particular shape
dimensional_array = torch.tensor((2,4))
print(dimensional_array)

tensor([2, 4])


FROM HERE WE  START WITH THE PYTORCH TUTORIAL

In [5]:
#from here we start with the data  from the pytorch tutorial
#IMPORT THE NECESSARY THE LIBRARIES
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [12]:
#here we import the data
train_data = datasets.FashionMNIST(
    root = "data",
    transform = ToTensor(),
    download = "False",
    train = True,
)

test_data = datasets.FashionMNIST(
    root = "data",
    transform = ToTensor(),
    download = "False",
    train = False,
)

In [13]:
print(len(train_data))
print(len(test_data))

60000
10000


In [16]:
#now lets create the data loader
batch_size = 64
train_dataloader = DataLoader(train_data , batch_size = batch_size , shuffle = True)
test_dataloader = DataLoader(test_data ,batch_size = batch_size, shuffle = True )

#now lets check for the shape of the train data loader
for X,y in train_dataloader:
  print(f"THE SHAPE OF THE INPUT TO THE MODEL SHOULD BE " , X.shape)
  print(f" THE SHAPE OF THE OUTPUT TO THE MODEL SHOULD BE " , y.shape)
  break


THE SHAPE OF THE INPUT TO THE MODEL SHOULD BE  torch.Size([64, 1, 28, 28])
 THE SHAPE OF THE OUTPUT TO THE MODEL SHOULD BE  torch.Size([64])


In [23]:
#now lets check for the accelerator if it present
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else 'cpu'
print(f"Using {device}")

#now lets define the model
#always inherits from the nn.Module

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten() #this flattens the data which in our case is 28x28
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28 , 512) ,
        nn.ReLU() ,
        nn.Linear(512 , 512),
        nn.ReLU() ,
        nn.Linear(512 , 10) #Linear takes in parameter as input_channels , output _channels
        #this has created the model , lets now take it a notch up and fit the data to the model

    )
  def forward(self , x):
    x = self.flatten(x)
    x = self.linear_relu_stack(x)
    return x
    #this returns  x after performing the above operations

neural_net = NeuralNetwork().to(device) #transfer everything to the required device
print(neural_net.parameters)


Using cpu
<bound method Module.parameters of NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)>


In [28]:
#now we need to also train the model  for this we use the loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(neural_net.parameters() , lr = 1e-3)

#now lets train the model
def train(dataloader ,model , loss_fn ,optimizer ):
  size = len(dataloader.dataset) #dataloader.dataset just returns  the  original dataset from ehich you set the  dataloader
  #now we train the model
  for batch , (X,y) in enumerate(dataloader):
    #transfer the things to the required device
    prediction = model(X) #this gives us the preediction of the model
    #after getting the prediction ,lets now get the loss and then train the model
    loss = loss_fn(prediction , y)
    #now for making the optimiser step lets fir make the gradient as 0
    optimizer.zero_grad()
    #now move backward to get the gradients
    loss.backward()
    #after getting the loss backward , now lets  use the optimizer step
    optimizer.step()

    #now we print the  metrics we recieved after every epoch
    if batch % 100 == 0:
      loss , current = loss.item() , (batch) * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader , model , loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  #now we put the model in the eval model
  model.eval()
  test_loss , correct = 0,0
  with torch.no_grad(): # for the testing part , the gradient need not be calculated
    for X,y in dataloader:
      X , y = X.to(device) , y.to(device)
      prediction = model(X)
      test_loss += loss_fn(prediction , y).item()
  test_loss = test_loss / num_batches
  print(f"Average loss is {test_loss:>8f} \n")


In [29]:
epochs  = 10
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader , neural_net , loss_fn , optimizer)
  test(test_dataloader , neural_net , loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.847936  [    0/60000]
loss: 0.830036  [ 6400/60000]
loss: 0.750630  [12800/60000]
loss: 0.771198  [19200/60000]
loss: 0.728050  [25600/60000]
loss: 0.752310  [32000/60000]
loss: 0.771431  [38400/60000]
loss: 0.786946  [44800/60000]
loss: 0.636334  [51200/60000]
loss: 0.717894  [57600/60000]
Average loss is 0.763005 

Epoch 2
-------------------------------
loss: 0.895226  [    0/60000]
loss: 0.865263  [ 6400/60000]
loss: 0.712641  [12800/60000]
loss: 0.941781  [19200/60000]
loss: 0.834143  [25600/60000]
loss: 0.648712  [32000/60000]
loss: 0.718914  [38400/60000]
loss: 0.878667  [44800/60000]
loss: 0.682874  [51200/60000]
loss: 0.822990  [57600/60000]
Average loss is 0.741653 

Epoch 3
-------------------------------
loss: 0.728123  [    0/60000]
loss: 0.855000  [ 6400/60000]
loss: 0.656147  [12800/60000]
loss: 0.571629  [19200/60000]
loss: 0.768432  [25600/60000]
loss: 0.719920  [32000/60000]
loss: 0.709778  [38400/60000]
loss: 0.704913  

In [39]:
#now lets save the model
torch.save(neural_net.state_dict() , "/content/data/FashionMNIST/neural_network_model1")
print(f"Saved PyTorch Model State to {"/content/data/FashionMNIST/neural_network_model1"}")

Saved PyTorch Model State to /content/data/FashionMNIST/neural_network_model1


In [40]:
#now lets load the model again to ensure that everything is okay

model.load_state_dict(torch.load("/content/data/FashionMNIST/neural_network_model1" , weights_only = True))
#if all the keys match perfectly that means that the model was successfully loaded


<All keys matched successfully>